In [1]:
# Import libraries, features and settings (not all of these are needed so pull what you need)

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import io
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn import preprocessing
plt.rc("font", size = 14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
sns.set(style="white")
sns.set(style="whitegrid", color_codes = True)

In [2]:
#reading the given csv
df = pd.read_csv("historic.csv")
# To display the top 5 rows
df.head(10)

,item_no,category,main_promotion,color,stars,success_indicator
0,739157,Tunic,Catalog,Green,3.1,flop
1,591846,Hoodie,Category_Highlight,Red,1.5,flop
2,337574,Sweatshirt,Catalog,Red,4.4,top
3,401933,Polo-Shirt,Category_Highlight,Blue,3.1,flop
4,812151,Hoodie,Category_Highlight,Green,4.1,top
5,200284,Hoodie,Display_Ad_Campaign,Yellow,3.9,flop
6,974264,Sweatshirt,Catalog,Red,1.4,flop
7,389059,Sweatshirt,Category_Highlight,Red,1.8,top
8,413025,T-Shirt,Catalog,Black,3.2,top
9,615692,Hoodie,Category_Highlight,White,5.0,top


In [3]:
#converting the given data into binary format of 0 and 1
df['success_indicator'] = df['success_indicator'].apply(lambda x: 0 if x == 'flop' else 1)
df

,item_no,category,main_promotion,color,stars,success_indicator
0,739157,Tunic,Catalog,Green,3.1,0
1,591846,Hoodie,Category_Highlight,Red,1.5,0
2,337574,Sweatshirt,Catalog,Red,4.4,1
3,401933,Polo-Shirt,Category_Highlight,Blue,3.1,0
4,812151,Hoodie,Category_Highlight,Green,4.1,1
...,...,...,...,...,...,...
7995,280947,T-Shirt,Catalog,Blue,3.9,0
7996,874952,Sweatshirt,Frontpage_Header,Green,3.5,1
7997,891640,T-Shirt,Category_Highlight,Yellow,4.0,0
7998,786448,Blouse,Category_Highlight,Blue,2.6,0


In [4]:
#Dropping unnecesary columns
df = df.drop(['item_no'], axis=1)
df.head(5)

,category,main_promotion,color,stars,success_indicator
0,Tunic,Catalog,Green,3.1,0
1,Hoodie,Category_Highlight,Red,1.5,0
2,Sweatshirt,Catalog,Red,4.4,1
3,Polo-Shirt,Category_Highlight,Blue,3.1,0
4,Hoodie,Category_Highlight,Green,4.1,1


In [8]:
df["category"].unique()

array([5, 1, 3, 2, 4, 0])

In [7]:
from sklearn.preprocessing import LabelEncoder
le_category = LabelEncoder()
df['category'] = le_category.fit_transform(df['category'])
df['category'].unique()

array([5, 1, 3, 2, 4, 0])

In [9]:
df["main_promotion"].unique()

array(['Catalog', 'Category_Highlight', 'Display_Ad_Campaign',
       'Frontpage_Header'], dtype=object)

In [10]:
le_main_promotion = LabelEncoder()
df['main_promotion'] = le_category.fit_transform(df['main_promotion'])
df['main_promotion'].unique()

array([0, 1, 2, 3])

In [11]:
df["color"].unique()

array(['Green', 'Red', 'Blue', 'Yellow', 'Black', 'White', 'Multi-Color',
       'Brown', 'Pink', 'Orange'], dtype=object)

In [12]:
le_color = LabelEncoder()
df['color'] = le_category.fit_transform(df['color'])
df['color'].unique()

array([3, 7, 1, 9, 0, 8, 4, 2, 6, 5])

In [ ]:
df

,item_no,category,main_promotion,color,stars,success_indicator
0,739157,5,0,3,3.1,0
1,591846,1,1,7,1.5,0
2,337574,3,0,7,4.4,1
3,401933,2,1,1,3.1,0
4,812151,1,1,3,4.1,1
...,...,...,...,...,...,...
7995,280947,4,0,1,3.9,0
7996,874952,3,3,3,3.5,1
7997,891640,4,1,9,4.0,0
7998,786448,0,1,1,2.6,0


In [13]:
#Import MinMax Scaler
from sklearn.preprocessing import MinMaxScaler

# initialize min-max scaler
mm_scaler = MinMaxScaler()
column_names = df.columns.tolist()
column_names.remove('success_indicator')

# Transform all attributes
df[column_names] = mm_scaler.fit_transform(df[column_names])
df.sort_index(inplace=True)
df.head()

,category,main_promotion,color,stars,success_indicator
0,1.0,0.000000,0.333333,0.482759,0
1,0.2,0.333333,0.777778,0.206897,0
2,0.6,0.000000,0.777778,0.706897,1
3,0.4,0.333333,0.111111,0.482759,0
4,0.2,0.333333,0.333333,0.655172,1


In [14]:
# Selecting the Feature, by remove the unused feature
feature = ['success_indicator']
train_feature = df.drop(feature, axis=1)

# Set The Target
train_target = df["success_indicator"]

In [15]:
# Show the Feature
train_feature.head(5)

,category,main_promotion,color,stars
0,1.0,0.000000,0.333333,0.482759
1,0.2,0.333333,0.777778,0.206897
2,0.6,0.000000,0.777778,0.706897
3,0.4,0.333333,0.111111,0.482759
4,0.2,0.333333,0.333333,0.655172


In [16]:
# Split Data
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(train_feature ,train_target, shuffle = True, test_size=0.3, random_state=1)

In [17]:
# Show the training data
X_train.head()

,category,main_promotion,color,stars
3941,0.8,0.333333,0.444444,0.655172
725,1.0,0.333333,0.222222,0.637931
102,0.8,1.000000,1.000000,0.672414
3680,0.8,0.000000,0.000000,0.482759
1357,0.6,1.000000,0.111111,0.482759


In [18]:
# Import Library
from sklearn.neural_network import MLPClassifier

# Fitting Model
mlp = MLPClassifier(hidden_layer_sizes=(5), activation = 'relu', solver = 'adam',max_iter= 10000, verbose = True)
mlp = mlp.fit(X_train,y_train)

# Prediction to Test Dataset
y_predmlp = mlp.predict(X_test)

Iteration 1, loss = 1.04409383
Iteration 2, loss = 0.96662536
Iteration 3, loss = 0.90032086
Iteration 4, loss = 0.84311085
Iteration 5, loss = 0.79375779
Iteration 6, loss = 0.75191504
Iteration 7, loss = 0.71786280
Iteration 8, loss = 0.69163499
Iteration 9, loss = 0.67199056
Iteration 10, loss = 0.65731485
Iteration 11, loss = 0.64635823
Iteration 12, loss = 0.63767028
Iteration 13, loss = 0.63048317
Iteration 14, loss = 0.62399613
Iteration 15, loss = 0.61810977
Iteration 16, loss = 0.61270306
Iteration 17, loss = 0.60771462
Iteration 18, loss = 0.60295472
Iteration 19, loss = 0.59841496
Iteration 20, loss = 0.59403605
Iteration 21, loss = 0.58967558
Iteration 22, loss = 0.58539287
Iteration 23, loss = 0.58119491
Iteration 24, loss = 0.57711851
Iteration 25, loss = 0.57315397
Iteration 26, loss = 0.56931363
Iteration 27, loss = 0.56555051
Iteration 28, loss = 0.56192752
Iteration 29, loss = 0.55831630
Iteration 30, loss = 0.55489464
Iteration 31, loss = 0.55171054
Iteration 32, los

In [19]:
print('Number of Layer =', mlp.n_layers_)
print('Number of Iteration =', mlp.n_iter_)
print('Current loss computed with the loss function =', mlp.loss_)

Number of Layer = 3
Number of Iteration = 186
Current loss computed with the loss function = 0.4741111815779447


In [20]:
# Import the metrics class
from sklearn import metrics


In [21]:
# Confussion Matrix
cnf_matrixmlp = metrics.confusion_matrix(y_test, y_predmlp)
cnf_matrixmlp

array([[ 507,  353],
       [ 134, 1406]])

In [22]:
# Show the Accuracy, Precision, Recall, F1, etc.
acc_mlp = metrics.accuracy_score(y_test, y_predmlp)
prec_mlp = metrics.precision_score(y_test, y_predmlp)
rec_mlp = metrics.recall_score(y_test, y_predmlp)
f1_mlp = metrics.f1_score(y_test, y_predmlp)
kappa_mlp = metrics.cohen_kappa_score(y_test, y_predmlp)

In [23]:

print("Accuracy:", acc_mlp)
print("Precision:", prec_mlp)
print("Recall:", rec_mlp)
print("F1 Score:", f1_mlp)
print("Cohens Kappa Score:", kappa_mlp)

Accuracy: 0.7970833333333334
Precision: 0.7993177942012507
Recall: 0.912987012987013
F1 Score: 0.8523795089421036
Cohens Kappa Score: 0.5324575579627822
